In [1]:
!rm -rf kits23


In [ ]:
#hansani

In [2]:
# === 0. Install dependencies ===
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q monai nibabel SimpleITK scikit-image scipy tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 26.8 MB/s eta 0:00:00


In [3]:
# === 1. Imports & utility functions ===
import os
import glob
import json
import nibabel as nib
import numpy as np
from scipy import ndimage
from tqdm import tqdm

In [4]:
# === 2. Paths & dataset JSON creation ===
# Adjust these paths to match how your dataset is mounted in Colab
DATA_ROOT = "/content/kits23/dataset"   # root folder containing imagesTr/ and labelsTr/
CROPPED_ROOT = "/content/kits23/cropped"
os.makedirs(CROPPED_ROOT, exist_ok=True)

In [5]:
# find image and label paths
images_tr = sorted(glob.glob(os.path.join(DATA_ROOT, "imagesTr", "*.nii*")) +
                   glob.glob(os.path.join(DATA_ROOT, "imagesTr", "*.nii.gz")))
labels_tr = sorted(glob.glob(os.path.join(DATA_ROOT, "labelsTr", "*.nii*")) +
                   glob.glob(os.path.join(DATA_ROOT, "labelsTr", "*.nii.gz")))

assert len(images_tr) == len(labels_tr), f"Mismatch images {len(images_tr)} vs labels {len(labels_tr)}"


In [6]:
# Build dataset.json relative to DATAROOT
datalist = {"training": []}
for img_p, lbl_p in zip(images_tr, labels_tr):
    rel_img = os.path.relpath(img_p, start=DATA_ROOT)
    rel_lbl = os.path.relpath(lbl_p, start=DATA_ROOT)
    datalist["training"].append({
        "image": rel_img,
        "label": rel_lbl,
        "image_filename": os.path.basename(img_p)
    })

dataset_json = {
    "name": "KiTS23",
    "description": "KiTS23 dataset for Auto3DSeg",
    "reference": "",
    "licence": "",
    "release": "0.0",
    "modality": {"0": "CT"},
    "labels": {"0": "background", "1": "kidney_and_mass", "2": "mass", "3": "tumor"},
    "numTraining": len(datalist["training"]),
    "training": datalist["training"],
    "test": []
}

with open(os.path.join(DATA_ROOT, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=2)
print("Wrote dataset.json with", len(datalist["training"]), "cases")



Wrote dataset.json with 0 cases


In [7]:
# === 3. Cropping strategy: coarse bounding box cropping (kidney region) ===
def crop_volume_to_kidney_bbox(img_path, lbl_path, pad=10):
    img = nib.load(img_path)
    lbl = nib.load(lbl_path)
    img_arr = img.get_fdata()
    lbl_arr = lbl.get_fdata()
    fg = lbl_arr > 0
    coords = np.array(np.nonzero(fg))
    if coords.size == 0:
        # nothing to crop
        return None, None, None
    mins = coords.min(axis=1)
    maxs = coords.max(axis=1)
    # pad
    mins = np.maximum(mins - pad, 0)
    maxs = np.minimum(maxs + pad, np.array(lbl_arr.shape) - 1)
    cropped_img = img_arr[mins[0]: maxs[0] + 1,
                          mins[1]: maxs[1] + 1,
                          mins[2]: maxs[2] + 1]
    cropped_lbl = lbl_arr[mins[0]: maxs[0] + 1,
                          mins[1]: maxs[1] + 1,
                          mins[2]: maxs[2] + 1]
    # create new affine mapping
    new_affine = img.affine.copy()
    # shift the origin: new_affine = original_affine + translation
    new_affine[:3, 3] += img.affine[:3, :3].dot(mins)
    return cropped_img, cropped_lbl, new_affine


In [8]:
print("Cropping all training volumes to kidney bounding boxes (with padding)...")
for img_p, lbl_p in tqdm(zip(images_tr, labels_tr), total=len(images_tr)):
    cropped_img, cropped_lbl, aff = crop_volume_to_kidney_bbox(img_p, lbl_p, pad=10)
    if cropped_img is None:
        continue
    # save cropped image and label as new NIfTI
    base = os.path.basename(img_p)
    out_img = nib.Nifti1Image(cropped_img.astype(np.float32), aff)
    out_lbl = nib.Nifti1Image(cropped_lbl.astype(np.uint8), aff)
    nib.save(out_img, os.path.join(CROPPED_ROOT, base))
    nib.save(out_lbl, os.path.join(CROPPED_ROOT, os.path.basename(lbl_p)))


Cropping all training volumes to kidney bounding boxes (with padding)...


0it [00:00, ?it/s]


In [19]:
# After cropping, you can choose to train your networks on cropped volumes by changing your datalist to point to CROPPED_ROOT instead of original.

# === 4. Prepare input.yaml for Auto3DSeg ===
input_yaml = {
    "modality": "CT",
    "datalist": "dataset.json",
    "dataroot": "./dataset",   # this is relative to working directory, adjust if needed
    "class_names": [
        {"name": "kidney_and_mass", "index": [1, 2, 3]},
        {"name": "mass", "index": [2, 3]},
        {"name": "tumor", "index": [2]}
    ],
    "sigmoid": True
}
with open(os.path.join(DATA_ROOT, "input.yaml"), "w") as f:
    json.dump(input_yaml, f, indent=2)
print("Wrote input.yaml")

Wrote input.yaml


In [14]:
!pip uninstall -y nni

Found existing installation: nni 3.0
Uninstalling nni-3.0:
  Successfully uninstalled nni-3.0


In [15]:
!pip install --upgrade filelock

In [16]:
!pip install -q "monai[all]" fire --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.31.7 requires filelock>=3.15, but you have filelock 3.11.0 which is incompatible.


In [17]:
# 1. Uninstall conflicting packages
!pip uninstall -y nni filelock pytensor

# 2. Upgrade pip to latest
!pip install --upgrade pip

# 3. Install MONAI with all optional dependencies + fire
!pip install "monai[all]" fire --upgrade

# 4. Verify versions
!pip show monai filelock fire


Found existing installation: nni 3.0
Uninstalling nni-3.0:
  Successfully uninstalled nni-3.0
Found existing installation: filelock 3.11.0
Uninstalling filelock-3.11.0:
  Successfully uninstalled filelock-3.11.0
Found existing installation: pytensor 2.31.7
Uninstalling pytensor-2.31.7:
  Successfully uninstalled pytensor-2.31.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Using cached nni-3.0-py3-none-manylinux1_x86_64.whl.metadata (19 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached filelock-3.11.0-py3-none-any.whl.metadata (2.5 kB)
Using cached nni-3.0-py3-none-manylinux1_x86_64.whl (61.4 MB)
Using cached filelock-3.11.0-py3-none-any.whl (10.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [nni]
Name: monai
Version: 1.5.1
Summary: AI Toolkit for Healthcare Imaging
Home-

2025-10-14 08:32:12.268535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760430732.695663   32199 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760430732.808702   32199 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760430733.679261   32199 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760430733.679356   32199 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760430733.679362   32199 computation_placer.cc:177] computation placer alr

In [20]:
# === 5. Run Auto3DSeg training + inference ===
# This will run all default algorithms (SegResNet, DiNTS, SwinUNETR) in 5-fold CV and ensemble
!python -m monai.apps.auto3dseg AutoRunner run --input "{os.path.join(DATA_ROOT, 'input.yaml')}"


2025-10-14 08:44:49.263103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760431489.288001   35586 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760431489.295287   35586 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760431489.314092   35586 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760431489.314144   35586 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760431489.314151   35586 computation_placer.cc:177] computation placer alr

In [ ]:
# === 6. Postprocessing the predictions ===
# After inference, the predictions are placed in a directory under the Auto3DSeg workdir (e.g. `ensemble_output` or `predictions`)
PRED_FOLDER = "/content/kits23/work_dir/ensemble_output"  # example — adjust to actual path
PRED_PP = "/content/kits23/pred_pp"
os.makedirs(PRED_PP, exist_ok=True)

for p in sorted(glob.glob(os.path.join(PRED_FOLDER, "*.nii*")) + glob.glob(os.path.join(PRED_FOLDER, "*.nii"))):
    pred_nii = nib.load(p)
    pred = pred_nii.get_fdata()
    # pred shape may be (C, Z, Y, X) if multi-channel
    # threshold to binary masks
    masks = (pred > 0.5).astype(np.uint8)

    # remove small components in merged foreground
    merged = np.any(masks, axis=0).astype(np.uint8)
    labeled_fg, ncomp = ndimage.label(merged)
    sizes = ndimage.sum(merged, labeled_fg, range(1, ncomp + 1))
    keep_mask = np.zeros_like(merged)
    for idx, s in enumerate(sizes, start=1):
        if s >= 100:
            keep_mask[labeled_fg == idx] = 1
    masks_clean = masks * keep_mask

    # tumor/cyst rim correction
    # Example: if cyst channel (index maybe 1) surrounds tumor channel (index 2), remove that rim.
    # This is task-specific; here is a naive approach:
    # Assume channel 2 is tumor, channel 1 is mass (cyst + tumor), etc. You should verify mapping.
    # e.g., remove pixels in cyst class that are one-voxel neighbors of tumor class
    tumor_mask = masks_clean[2]
    # dilate tumor and subtract overlap from cyst mask
    struct = np.ones((3, 3, 3))
    dil = ndimage.binary_dilation(tumor_mask, structure=struct)
    cyst_mask = masks_clean[1]
    cyst_mask[dil & (cyst_mask == 1)] = 0
    masks_clean[1] = cyst_mask

    # Save cleaned prediction (multi-channel) or merge into integer-label file
    out_affine = pred_nii.affine
    # e.g. convert multi-channel masks_clean to a single integer map:
    # label 1: kidney_and_mass, label 2: mass, label 3: tumor (or whichever your submission expects)
    # simple merging: prioritize tumor, then mass, then kidney
    final = np.zeros_like(merged, dtype=np.uint8)
    # e.g. assign tumor
    final[tumor_mask == 1] = 3
    final[cyst_mask == 1] = 2
    # kidney_and_mass = background + kidney region mask
    # final[merged == 1] = 1  # or whatever your mapping
    # (you’ll need to adjust mapping to match KiTS expected submission format)

    nib.save(nib.Nifti1Image(final, out_affine), os.path.join(PRED_PP, os.path.basename(p)))

print("Postprocessing done, saved to", PRED_PP)

# === 7. (Optional) Evaluate using KiTS metric script ===
# If you have the official KiTS23 metric tool installed/available:
# !python kits23_compute_metrics {PRED_PP} -num_processes 8